# Tarea 2 Manipulación de Datos
### Mauricio Sierra 2076860

**Resumen:** Se hizo un webscrapping de la Página de ESPN para sacar los salarios de los jugadores de la NBA para la temporada 24-25. Adicional a esto se hizo un merge con una base de datos proporcionada por StatHead sobre las estadisticas de estos mismos.

Los datos se modificaron para quitar caracteres raros y mejorar los resultados encontrados en el merge, además modifique las variables y genere nuevas para terminar con un dataset para el proyecto Final

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

Salary_list = []
for i in range(1,14):
    
    url = f"https://www.espn.com/nba/salaries/_/year/2025/page/{i}"
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    # Encuentra la tabla
    table = soup.find("table")
    rows = table.find_all("tr")[1:]  # omite encabezado

    for row in rows:
        cols = [col.text.strip() for col in row.find_all("td")]
        Salary_list.append(cols)

Salary = pd.DataFrame(Salary_list, columns=["Rank","Player", "Team", "Salary"])
Salary.to_csv('C:/Users/Mauricio/Documents/Maestría en Ciencia de Datos/Segundo Tetramestre/Aprendizaje Automatico/Salarys.csv')
Salary.head()

,Rank,Player,Team,Salary
0,1,"Stephen Curry, G",Golden State Warriors,"$55,761,216"
1,2,"Joel Embiid, C",Philadelphia 76ers,"$51,415,938"
2,3,"Nikola Jokic, C",Denver Nuggets,"$51,415,938"
3,4,"Kevin Durant, F",Phoenix Suns,"$51,179,021"
4,5,"Bradley Beal, G",Phoenix Suns,"$50,203,930"


In [2]:
import re
def Clean_Name_Salary(nombre):
    nombre = re.sub(r'\b(Sr\.)\b', '', nombre)
    return nombre.strip()

def Clean_Name_Og(nombre):
    nombre = re.sub(r'(?<!Jr)\.', '', nombre)
    return nombre.strip()

In [3]:
Salary[["Player", "Pos"]] = Salary["Player"].str.split(",", expand=True)
Salary["Player"] = Salary["Player"].str.strip()
Salary["Pos"] = Salary["Pos"].str.strip()
Salary["Player"] = Salary["Player"].apply(Clean_Name_Salary)
Salary.to_csv('C:/Users/Mauricio/Documents/Maestría en Ciencia de Datos/Segundo Tetramestre/Aprendizaje Automatico/Salarys.csv')
Salary.head()

,Rank,Player,Team,Salary,Pos
0,1,Stephen Curry,Golden State Warriors,"$55,761,216",G
1,2,Joel Embiid,Philadelphia 76ers,"$51,415,938",C
2,3,Nikola Jokic,Denver Nuggets,"$51,415,938",C
3,4,Kevin Durant,Phoenix Suns,"$51,179,021",F
4,5,Bradley Beal,Phoenix Suns,"$50,203,930",G


In [4]:
from unidecode import unidecode
NBA_DB = pd.read_csv('C:/Users/Mauricio/Documents/Maestría en Ciencia de Datos/Segundo Tetramestre/Aprendizaje Automatico/NBA Original Dataset.csv')
NBA_DB["Player"] = NBA_DB["Player"].apply(unidecode)
NBA_DB["Player"] = NBA_DB["Player"].apply(Clean_Name_Og)
NBA_DB.head()

,Rk,Player,WS,USG%,BPM,VORP,PER,Season,Age,Team,...,TOV,PF,PTS,FG%,2P%,3P%,FT%,TS%,eFG%,Pos
0,1,Shai Gilgeous-Alexander,16.7,34.8,11.5,8.9,30.7,2024-25,26,OKC,...,3.4,3.0,45.9,0.519,0.571,0.375,0.898,0.637,0.569,G
1,2,Nikola Jokic,16.4,29.5,13.3,9.8,32.0,2024-25,29,DEN,...,4.3,3.0,38.7,0.576,0.627,0.417,0.800,0.663,0.627,C-F
2,3,Ivica Zubac,11.7,19.5,3.1,3.4,22.3,2024-25,27,LAC,...,2.3,3.1,24.5,0.628,0.628,NaN,0.661,0.641,0.628,C
3,4,Jarrett Allen,11.6,15.9,4.2,3.6,22.1,2024-25,26,CLE,...,2.0,2.7,23.1,0.706,0.712,0.000,0.718,0.724,0.706,C-F
4,5,Giannis Antetokounmpo,11.5,35.2,9.5,6.6,30.5,2024-25,30,MIL,...,4.3,3.3,42.7,0.601,0.620,0.222,0.617,0.625,0.607,F-G


In [5]:
Final_DB = pd.merge(NBA_DB,Salary[['Player','Salary']],on='Player',how='left')
Final_DB = Final_DB.dropna()
Final_DB.to_csv('C:/Users/Mauricio/Documents/Maestría en Ciencia de Datos/Segundo Tetramestre/Aprendizaje Automatico/Final_DB.csv')
Final_DB.head()

,Rk,Player,WS,USG%,BPM,VORP,PER,Season,Age,Team,...,PF,PTS,FG%,2P%,3P%,FT%,TS%,eFG%,Pos,Salary
0,1,Shai Gilgeous-Alexander,16.7,34.8,11.5,8.9,30.7,2024-25,26,OKC,...,3.0,45.9,0.519,0.571,0.375,0.898,0.637,0.569,G,"$35,859,950"
1,2,Nikola Jokic,16.4,29.5,13.3,9.8,32.0,2024-25,29,DEN,...,3.0,38.7,0.576,0.627,0.417,0.800,0.663,0.627,C-F,"$51,415,938"
3,4,Jarrett Allen,11.6,15.9,4.2,3.6,22.1,2024-25,26,CLE,...,2.7,23.1,0.706,0.712,0.000,0.718,0.724,0.706,C-F,"$20,000,000"
4,5,Giannis Antetokounmpo,11.5,35.2,9.5,6.6,30.5,2024-25,30,MIL,...,3.3,42.7,0.601,0.620,0.222,0.617,0.625,0.607,F-G,"$48,787,676"
5,6,Tyrese Haliburton,10.4,21.6,5.8,4.9,21.8,2024-25,24,IND,...,1.8,26.6,0.473,0.581,0.388,0.851,0.616,0.582,G,"$42,176,000"


In [6]:
M_Final_DB = Final_DB[["Player","WS","USG%","BPM","VORP","PER","Age","G","2PA","2P%","3PA","3P%","FTA","FT%","ORB","DRB","AST","STL","BLK","TOV","PF","PTS","Pos","Salary"]]
M_Final_DB['Reliability'] = M_Final_DB["G"]/82
Final_DB.to_csv('C:/Users/Mauricio/Documents/Maestría en Ciencia de Datos/Segundo Tetramestre/Aprendizaje Automatico/Final_DB_Modificado.csv')
M_Final_DB.head()

C:\Users\Mauricio\AppData\Local\Temp\ipykernel_13144\572303476.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  M_Final_DB['Reliability'] = M_Final_DB["G"]/82


,Player,WS,USG%,BPM,VORP,PER,Age,G,2PA,2P%,...,DRB,AST,STL,BLK,TOV,PF,PTS,Pos,Salary,Reliability
0,Shai Gilgeous-Alexander,16.7,34.8,11.5,8.9,30.7,26,76,22.6,0.571,...,5.8,9.0,2.4,1.4,3.4,3.0,45.9,G,"$35,859,950",0.926829
1,Nikola Jokic,16.4,29.5,13.3,9.8,32.0,29,70,19.3,0.627,...,12.9,13.4,2.4,0.8,4.3,3.0,38.7,C-F,"$51,415,938",0.853659
3,Jarrett Allen,11.6,15.9,4.2,3.6,22.1,26,82,13.3,0.712,...,12.2,3.3,1.6,1.5,2.0,2.7,23.1,C-F,"$20,000,000",1.000000
4,Giannis Antetokounmpo,11.5,35.2,9.5,6.6,30.5,30,67,26.3,0.620,...,13.7,9.1,1.2,1.6,4.3,3.3,42.7,F-G,"$48,787,676",0.817073
5,Tyrese Haliburton,10.4,21.6,5.8,4.9,21.8,24,73,8.7,0.581,...,4.2,13.2,2.1,1.0,2.4,1.8,26.6,G,"$42,176,000",0.890244
